In [1]:
import pandas as pd

In [2]:
cleaned_df = pd.read_csv("cleaned_news.csv")

In [5]:
cleaned_df.shape

(5923, 3)

In [6]:
def analyze_dataframe(df):
    total_records = len(df)
    duplicates_count = df.duplicated().sum()
    null_storytext_count = df['storytext'].isnull().sum()
    cleaned_df = df.drop_duplicates().dropna(subset=['storytext'])
    remaining_records = len(cleaned_df)

    # Summary table
    summary_table = pd.DataFrame({
        'Total Records': [total_records],
        'Duplicates': [duplicates_count],
        'Null "storytext"': [null_storytext_count],
        'Remaining Records': [remaining_records]
    })

    return cleaned_df

In [7]:
cleaned_df = analyze_dataframe(cleaned_df)

In [8]:
cleaned_df.shape

(3209, 3)

In [9]:
filtered_df = cleaned_df[cleaned_df['cleantext'].str.len() <= 2500]

In [10]:
import re
filtered_df['cleantext'] = filtered_df['cleantext'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

C:\Users\gnammi\AppData\Local\Temp\ipykernel_28968\2625654042.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['cleantext'] = filtered_df['cleantext'].apply(lambda x: re.sub(r'[^\w\s]', '', x))


In [29]:
import httpx

In [35]:
# def extract_summary(df, text_column_name,output_file):
#     summary_results = []
#     for index, row in df.iterrows():
#         text = row[text_column_name]
#         if len(text) <= 2500:
#             url = f"http://10.220.0.85:30066/summary/periscope?story={text}"
#             response = httpx.post(url)
#             if response.status_code == 200:
#                 summary_json = response.json()
#                 summary_results.append(summary_json)
#             else:
#                 print(f"Error: Request for '{text}' failed with status code {response.status_code}")
    
   
#     with open(output_file, 'w') as f:
#         json.dump(summary_results, f)

#     print(f"summary results saved to '{output_file}'")

In [49]:
filtered_df = filtered_df[:100]

In [50]:
filtered_df.shape

(100, 3)

In [51]:
import httpx

def extract_summary(df, text_column_name, output_file):
    summary_results = []
    timeout = 10 

    for index, row in df.iterrows():
        text = row[text_column_name]
        if len(text) <= 2500:
            url = f"http://10.220.0.85:30066/summary/periscope?story={text}"
            try:
                response = httpx.post(url, timeout=timeout)
                if response.status_code == 200:
                    summary_json = response.json()
                    summary_results.append(summary_json)
                else:
                    print(f"Error: Request for '{text}' failed with status code {response.status_code}")
            except httpx.ReadTimeout:
                print(f"Error: Request for '{text}' timed out after {timeout} seconds")
            except Exception as e:
                print(f"Error: An exception occurred while processing '{text}': {e}")

    with open(output_file, 'w') as f:
        json.dump(summary_results, f)

    print(f"Summary results saved to '{output_file}'")

In [ ]:
extract_summary(filtered_df, 'cleantext', 'summary_extraction.json')